In [279]:
import time
import math
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
import numpy as np
import pandas as pd
import sklearn.preprocessing as prep

In [280]:
df = pd.read_csv('^NSEI_withoutnull.csv')
df.head()

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date,Open,High,Low,Close,Future_Close
0,2007-11-22,5564.649902,5608.649902,5394.350098,5519.350098,5608.600098
1,2007-11-23,5524.299805,5638.600098,5514.750000,5608.600098,5731.700195
2,2007-11-26,5611.299805,5772.549805,5608.950195,5731.700195,5698.149902
3,2007-11-27,5729.250000,5743.549805,5655.600098,5698.149902,5617.549805
4,2007-11-28,5699.549805,5749.950195,5595.500000,5617.549805,5634.600098


In [281]:
df = df.dropna(axis=0, how='any')
df.shape

(2418, 6)

In [282]:
df = df.drop(['Date'], axis=1)
df.head()

,Open,High,Low,Close,Future_Close
0,5564.649902,5608.649902,5394.350098,5519.350098,5608.600098
1,5524.299805,5638.600098,5514.750000,5608.600098,5731.700195
2,5611.299805,5772.549805,5608.950195,5731.700195,5698.149902
3,5729.250000,5743.549805,5655.600098,5698.149902,5617.549805
4,5699.549805,5749.950195,5595.500000,5617.549805,5634.600098


In [283]:
df['Future_Close'] = df[' Future_Close'] 
df.head()

,Open,High,Low,Close,Future_Close,Future_Close
0,5564.649902,5608.649902,5394.350098,5519.350098,5608.600098,5608.600098
1,5524.299805,5638.600098,5514.750000,5608.600098,5731.700195,5731.700195
2,5611.299805,5772.549805,5608.950195,5731.700195,5698.149902,5698.149902
3,5729.250000,5743.549805,5655.600098,5698.149902,5617.549805,5617.549805
4,5699.549805,5749.950195,5595.500000,5617.549805,5634.600098,5634.600098


In [284]:
df = df.drop([' Future_Close'], axis=1)
df.head()

,Open,High,Low,Close,Future_Close
0,5564.649902,5608.649902,5394.350098,5519.350098,5608.600098
1,5524.299805,5638.600098,5514.750000,5608.600098,5731.700195
2,5611.299805,5772.549805,5608.950195,5731.700195,5698.149902
3,5729.250000,5743.549805,5655.600098,5698.149902,5617.549805
4,5699.549805,5749.950195,5595.500000,5617.549805,5634.600098


### New Column 'Rat' added = Future_Close/Current_Close

In [285]:
df['Rat'] = df['Future_Close']/df['Close']

In [286]:
df['Rat']

0       1.016170
1       1.021948
2       0.994147
3       0.985855
4       1.003035
5       1.022743
6       1.017743
7       0.998866
8       1.013937
9       1.002475
10      1.003291
11      0.997707
12      1.022926
13      1.010177
14      0.983570
15      0.998283
16      0.955239
17      0.993993
18      1.001541
19      1.002669
20      1.037909
21      1.014311
22      1.001771
23      0.999704
24      1.009688
25      1.000937
26      1.005704
27      0.999862
28      1.015497
29      1.000765
          ...   
2388    1.000907
2389    1.002823
2390    0.996790
2391    1.011177
2392    1.007037
2393    1.006236
2394    1.000352
2395    0.997694
2396    0.997454
2397    1.002244
2398    1.008587
2399    1.004706
2400    0.997994
2401    1.003933
2402    0.997264
2403    1.010179
2404    0.998400
2405    1.002753
2406    0.999933
2407    0.990274
2408    0.995459
2409    1.000563
2410    1.001242
2411    0.990622
2412    0.996249
2413    0.993271
2414    1.009557
2415    1.0067

In [287]:
df['Rat'] = df['Rat']>1
df['Rat'] = df['Rat'].astype(float)
df.head()

,Open,High,Low,Close,Future_Close,Rat
0,5564.649902,5608.649902,5394.350098,5519.350098,5608.600098,1.0
1,5524.299805,5638.600098,5514.750000,5608.600098,5731.700195,1.0
2,5611.299805,5772.549805,5608.950195,5731.700195,5698.149902,0.0
3,5729.250000,5743.549805,5655.600098,5698.149902,5617.549805,0.0
4,5699.549805,5749.950195,5595.500000,5617.549805,5634.600098,1.0


#### we're dropping future_close since there can be only one dependent variable

In [288]:
df = df.drop(['Future_Close'], axis=1)
df.head()

,Open,High,Low,Close,Rat
0,5564.649902,5608.649902,5394.350098,5519.350098,1.0
1,5524.299805,5638.600098,5514.750000,5608.600098,1.0
2,5611.299805,5772.549805,5608.950195,5731.700195,0.0
3,5729.250000,5743.549805,5655.600098,5698.149902,0.0
4,5699.549805,5749.950195,5595.500000,5617.549805,1.0


In [289]:
from keras.models import Sequential
from keras.layers import Dense

np.random.seed(7)

In [290]:
df.shape

(2418, 5)

In [291]:
df_target = pd.get_dummies(df['Rat'])
df_new = pd.concat([df, df_target], axis=1)
df_new = df_new.drop(['Rat'], axis=1)
df = df_new
df.head()

,Open,High,Low,Close,0.0,1.0
0,5564.649902,5608.649902,5394.350098,5519.350098,0,1
1,5524.299805,5638.600098,5514.750000,5608.600098,0,1
2,5611.299805,5772.549805,5608.950195,5731.700195,1,0
3,5729.250000,5743.549805,5655.600098,5698.149902,1,0
4,5699.549805,5749.950195,5595.500000,5617.549805,0,1


In [292]:
df.shape

(2418, 6)

In [293]:
train_start=0
train_end=int(np.floor(0.8*df.shape[0]))
test_start=train_end+1
test_end=int(df.shape[0])
df = df.values
data_train=df[np.arange(train_start, train_end),:]
data_test=df[np.arange(test_start,test_end),:]
data_train=pd.DataFrame(data_train)
data_test=pd.DataFrame(data_test)

In [294]:
print(data_train.shape)
data_train.head()


(1934, 6)


,0,1,2,3,4,5
0,5564.649902,5608.649902,5394.350098,5519.350098,0.0,1.0
1,5524.299805,5638.600098,5514.750000,5608.600098,0.0,1.0
2,5611.299805,5772.549805,5608.950195,5731.700195,1.0,0.0
3,5729.250000,5743.549805,5655.600098,5698.149902,1.0,0.0
4,5699.549805,5749.950195,5595.500000,5617.549805,0.0,1.0


In [295]:
print(data_test.shape)
data_test.head()

(483, 6)


,0,1,2,3,4,5
0,7958.149902,7972.149902,7934.149902,7954.899902,1.0,0.0
1,7976.700195,7979.299805,7910.799805,7931.350098,1.0,0.0
2,7902.299805,7912.299805,7853.299805,7864.149902,1.0,0.0
3,7817.600098,7821.399902,7775.700195,7781.899902,1.0,0.0
4,7816.549805,7825.399902,7746.049805,7765.399902,1.0,0.0


In [296]:
x_train=data_train.iloc[:,0:4]
y_train=data_train.iloc[:, 4:6]
x_test=data_test.iloc[:,0:4]
y_test=data_test.iloc[:, 4:6]
features = 4

In [297]:
y_train.head()

,4,5
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,1.0,0.0
4,0.0,1.0


In [298]:
import tensorflow as tf
X = tf.placeholder(dtype=tf.float32, shape=[None, features])
Y = tf.placeholder(dtype=tf.float32, shape=[None, 2])

In [299]:
n_neurons_1 = 15
n_neurons_2 = 10
n_neurons_3 = 5
n_target = 2
sigma = 1
training_epochs = 1000
display_step = 1

In [300]:
import keras
weight_initializer = keras.initializers.VarianceScaling(mode="fan_avg", distribution="uniform", scale=sigma)
bias_initializer = tf.zeros_initializer()

In [301]:
W_hidden_1 = tf.Variable(weight_initializer([features, n_neurons_1]))
bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))
W_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))
W_hidden_3 = tf.Variable(weight_initializer([n_neurons_2, n_neurons_3]))
bias_hidden_3 = tf.Variable(bias_initializer([n_neurons_3]))
W_out = tf.Variable(weight_initializer([n_neurons_3, n_target]))
bias_out = tf.Variable(bias_initializer([n_target]))
hidden_1 = tf.nn.relu(tf.add(tf.matmul(X, W_hidden_1), bias_hidden_1))
hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, W_hidden_2), bias_hidden_2))
hidden_3 = tf.nn.relu(tf.add(tf.matmul(hidden_2, W_hidden_3), bias_hidden_3))
pred = (tf.add(tf.matmul(hidden_3, W_out), bias_out))

In [302]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

In [303]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(x_train)/500)
        X_batches = np.array_split(x_train, total_batch)
        Y_batches = np.array_split(y_train, total_batch)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = X_batches[i], Y_batches[i]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={X: batch_x,
                                                          Y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: x_test, Y: y_test}))
    global result 
    result = tf.argmax(pred, 1).eval({X: x_test, Y: y_test})

Epoch: 0001 cost= 326.012369792
Epoch: 0002 cost= 281.664321899
Epoch: 0003 cost= 238.836547852
Epoch: 0004 cost= 197.458374023
Epoch: 0005 cost= 159.245864868
Epoch: 0006 cost= 123.743288676
Epoch: 0007 cost= 89.645726522
Epoch: 0008 cost= 56.490753174
Epoch: 0009 cost= 23.741420110
Epoch: 0010 cost= 10.624919256
Epoch: 0011 cost= 27.332158407
Epoch: 0012 cost= 29.995756785
Epoch: 0013 cost= 21.780987422
Epoch: 0014 cost= 6.568208694
Epoch: 0015 cost= 9.546585083
Epoch: 0016 cost= 15.020848274
Epoch: 0017 cost= 11.211067518
Epoch: 0018 cost= 3.356939316
Epoch: 0019 cost= 7.053010464
Epoch: 0020 cost= 4.517140865
Epoch: 0021 cost= 3.755902688
Epoch: 0022 cost= 4.409454584
Epoch: 0023 cost= 2.586132367
Epoch: 0024 cost= 2.597632885
Epoch: 0025 cost= 2.653435787
Epoch: 0026 cost= 1.810789347
Epoch: 0027 cost= 2.295428554
Epoch: 0028 cost= 1.572273890
Epoch: 0029 cost= 1.433010499
Epoch: 0030 cost= 1.520868142
Epoch: 0031 cost= 1.501495083
Epoch: 0032 cost= 1.422719479
Epoch: 0033 cost= 1

Epoch: 0284 cost= 1.415912509
Epoch: 0285 cost= 1.285072287
Epoch: 0286 cost= 1.413837870
Epoch: 0287 cost= 1.282979210
Epoch: 0288 cost= 1.411695083
Epoch: 0289 cost= 1.280882994
Epoch: 0290 cost= 1.409615477
Epoch: 0291 cost= 1.278859496
Epoch: 0292 cost= 1.407592018
Epoch: 0293 cost= 1.276819110
Epoch: 0294 cost= 1.405505339
Epoch: 0295 cost= 1.274749955
Epoch: 0296 cost= 1.403426250
Epoch: 0297 cost= 1.272696892
Epoch: 0298 cost= 1.401330312
Epoch: 0299 cost= 1.270647128
Epoch: 0300 cost= 1.399266481
Epoch: 0301 cost= 1.268621802
Epoch: 0302 cost= 1.397224824
Epoch: 0303 cost= 1.266581655
Epoch: 0304 cost= 1.395143231
Epoch: 0305 cost= 1.264564355
Epoch: 0306 cost= 1.393101811
Epoch: 0307 cost= 1.262540738
Epoch: 0308 cost= 1.391030510
Epoch: 0309 cost= 1.260514299
Epoch: 0310 cost= 1.388971488
Epoch: 0311 cost= 1.258497993
Epoch: 0312 cost= 1.386901736
Epoch: 0313 cost= 1.256465872
Epoch: 0314 cost= 1.384833574
Epoch: 0315 cost= 1.254460375
Epoch: 0316 cost= 1.382802169
Epoch: 031

Epoch: 0571 cost= 1.040272077
Epoch: 0572 cost= 1.149995744
Epoch: 0573 cost= 1.038916866
Epoch: 0574 cost= 1.148424009
Epoch: 0575 cost= 1.037600537
Epoch: 0576 cost= 1.146879673
Epoch: 0577 cost= 1.036303461
Epoch: 0578 cost= 1.145352483
Epoch: 0579 cost= 1.034997682
Epoch: 0580 cost= 1.143813054
Epoch: 0581 cost= 1.033682088
Epoch: 0582 cost= 1.142245114
Epoch: 0583 cost= 1.032371879
Epoch: 0584 cost= 1.140714089
Epoch: 0585 cost= 1.031078756
Epoch: 0586 cost= 1.139181475
Epoch: 0587 cost= 1.029788454
Epoch: 0588 cost= 1.137658119
Epoch: 0589 cost= 1.028514385
Epoch: 0590 cost= 1.136157811
Epoch: 0591 cost= 1.027242482
Epoch: 0592 cost= 1.134627263
Epoch: 0593 cost= 1.025957882
Epoch: 0594 cost= 1.133120418
Epoch: 0595 cost= 1.024697959
Epoch: 0596 cost= 1.131617407
Epoch: 0597 cost= 1.023432970
Epoch: 0598 cost= 1.130099078
Epoch: 0599 cost= 1.022168974
Epoch: 0600 cost= 1.128591518
Epoch: 0601 cost= 1.020906250
Epoch: 0602 cost= 1.127089679
Epoch: 0603 cost= 1.019669831
Epoch: 060

Epoch: 0846 cost= 0.973707537
Epoch: 0847 cost= 0.900224725
Epoch: 0848 cost= 0.971488178
Epoch: 0849 cost= 0.899363160
Epoch: 0850 cost= 0.970845878
Epoch: 0851 cost= 0.899353683
Epoch: 0852 cost= 0.971375187
Epoch: 0853 cost= 0.898714105
Epoch: 0854 cost= 0.969755034
Epoch: 0855 cost= 0.896994710
Epoch: 0856 cost= 0.967572033
Epoch: 0857 cost= 0.895751377
Epoch: 0858 cost= 0.966346661
Epoch: 0859 cost= 0.895295481
Epoch: 0860 cost= 0.965188404
Epoch: 0861 cost= 0.895134509
Epoch: 0862 cost= 0.965153734
Epoch: 0863 cost= 0.895225426
Epoch: 0864 cost= 0.965580722
Epoch: 0865 cost= 0.894396881
Epoch: 0866 cost= 0.963766555
Epoch: 0867 cost= 0.892636061
Epoch: 0868 cost= 0.961632113
Epoch: 0869 cost= 0.891512930
Epoch: 0870 cost= 0.959919095
Epoch: 0871 cost= 0.891420345
Epoch: 0872 cost= 0.960165560
Epoch: 0873 cost= 0.891771992
Epoch: 0874 cost= 0.960814516
Epoch: 0875 cost= 0.890922387
Epoch: 0876 cost= 0.958848516
Epoch: 0877 cost= 0.888969660
Epoch: 0878 cost= 0.956494431
Epoch: 087